In [38]:
import cvxpy as cp
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection  import train_test_split
import matplotlib.pyplot as plt

In [141]:
import numpy as np
from tqdm import tqdm
# from sklearn.metrics.pairwise import rbf_kernel
class SVMCust:
    def __init__(self) -> None:
        theta=1
        self.W=0
        self.b=0
        self.a=0
        self.C=0
        self.m=0
        self.f=0
    
    def kg(self,a, b, sigma=1):
        # return np.exp((-(np.linalg.norm(a-b)**2))/(2*sigma**2))
        # print("dot:",np.dot(a,b))
        # # val=np.linalg.det(np.dot(a,b))
        a=a.reshape((self.f,1))
        b=b.reshape((self.f,1))
        val=np.matmul(a.T,b)
        # print("as",a.shape, "b:",b.shape)
        print(val.shape)
        return val
    # generating kernal matrix
    def k_mat(self,x, k_func=kg):
        m = x.shape[0]
        mat = np.zeros((m, m))
        for i in range(m):
            for j in range(m):
                mat[i, j] = self.kg(x[i, :], x[j, :])
        return mat

    def getAlphas(self):
        return self.a
    def getWeight(self):
        return self.W
    def getBias(self):
        return self.b
    def fit(self,X_train,y_train):
        print("shape:",X_train.shape)
        y_train[y_train==0]=-1
        (m,f)=X_train.shape
        self.f=f
        y_train=np.asarray(y_train)
        self.m=m
        X=np.array(X_train)
        # k = rbf_kernel(X)
        k=self.k_mat(X)
        print(np.all(np.abs(k-k.T) < 1e-08))
        # print(k)
        G=np.zeros((m,m))
        print("shaope:",G.shape)
        print("m:",m)
        for i in tqdm(range(0,m)):
            for j in range(0,m-i):
                # print("i:",i,"j:",j)
                # print("s:",(y[i]*(np.inner(X[i],np.transpose(X[j])))*y[j]).shape)
                G[i][j]=(y_train[i]*y_train[j]*(k[i][j]))
                G[j][i]=G[i][j]
        # G=y*k*np.transpose(y)
       

        self.a = cp.Variable(m)
        self.C = cp.Parameter(nonneg=True)
        self.C.value = 0.02
        print("min eigne val:",np.linalg.eigvals(G).min())
        # G = cp.atoms.affine.wraps.psd_wrap(G)
        obj = cp.Minimize( (0.5) * cp.quad_form(self.a,G)-np.ones((1,m))@ self.a)
        constraints = [0<=self.a,self.a<=self.C ,np.transpose(y_train)@self.a==0]
        prob = cp.Problem(obj, constraints)
        print("Is prob dcp:",prob.is_dcp())
        result = prob.solve()
        print(self.a.value)
        self.W=np.zeros(f,dtype='float64')
        tempa=np.array(self.a.value)

        print(tempa)
        for i in range(m):
            if tempa[i]==self.C.value:
                self.W+=y_train[i]* tempa[i]* X[i]
        cnt_pos_vec=0
        # print("W:",type(W))
        self.W=np.asarray(self.W,dtype='float64')
        for i in range(m):
            if tempa[i]==self.C.value:
                # print(X[i].dtype)
                
                # print("y:",self.W.dtype)
                # print("h",y.dtype)
                # print((self.W* np.transpose(X[i])).shape)
                self.b+=y_train[i]-np.matmul(self.W, np.transpose(X[i]))
                cnt_pos_vec+=1
        self.b/=max(1,cnt_pos_vec)
    def predict(self,X):
        # ycap=np.matmul(self.W,np.transpose(X))
        matb=np.full((X.shape[0],1),self.b)
        matw=np.full((X.shape[0],X.shape[1]),self.W)
        # print(matb)
        # print(matw)
        # print("shapE,",np.matmul(X,np.transpose(self.W)).shape)
        print(np.matmul(X,np.transpose(self.W)))
        # print("fnal:,",np.add(matb,np.matmul(X,np.transpose(self.W))).shape)
        y_cap=self.b+np.transpose(np.matmul(X,np.transpose(self.W)))
        print(y_cap.shape)
        y_cap[y_cap<=0]=-1
        y_cap[y_cap>0]=1
        return y_cap
        
        
        

        
        
        



    
    
    
    

In [142]:
import pandas as pd
import numpy as np
from sklearn import svm
import math

In [143]:
##global variables
tz=7
tpr_by_fpr_min_thresh=0

tc=400
target_column_name='income'

In [144]:
data=pd.read_csv("datasets/adult.csv")

In [145]:
data['income'].replace(['<=50K', '>50K'],
                        [0, 1], inplace=True)

In [146]:
list_of_catagorical_vars=['workclass','education',
       'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country',
]


In [147]:
from sklearn.preprocessing import LabelEncoder
encoded_to_orig_dict={}
for col_name in list_of_catagorical_vars:
    le = LabelEncoder()
    le.fit(data[col_name])
    data[col_name]=le.transform(data[col_name])
    

    encoded_to_orig_dict[col_name]= dict((v,k) for k,v in dict(zip(le.classes_, le.transform(le.classes_))).items())

In [148]:
traindt=data;
test=data[target_column_name]
traindt.drop(target_column_name,axis=1,inplace=True)


In [149]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(traindt,test, test_size=0.25) # 70% training and 30% test

In [150]:
clf=SVMCust()

In [151]:
clf.fit(X_train[:5000], y_train[:5000])

shape: (5000, 14)
(14, 14)


ValueError: setting an array element with a sequence.